In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

2023-11-30 18:19:08.349834: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-30 18:19:08.452253: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-30 18:19:08.794348: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/zach/miniconda3/envs/tf/lib/:/home/zach/miniconda3/envs/tf/lib/python3.10/site-packages/nvidia/cudnn/lib
2023-11-30 18:19:08.794390: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could no

### Squeezenet

In [2]:
%pip install git+https://github.com/rcmalli/keras-squeezenet.git

/bin/bash: /home/zach/miniconda3/envs/tf/lib/libtinfo.so.6: no version information available (required by /bin/bash)
  Cloning https://github.com/rcmalli/keras-squeezenet.git to /tmp/pip-req-build-xksbhphy
  Running command git clone --filter=blob:none --quiet https://github.com/rcmalli/keras-squeezenet.git /tmp/pip-req-build-xksbhphy
  Resolved https://github.com/rcmalli/keras-squeezenet.git to commit 4fb9cb7510ea0315303090edbc1bd97c2916af81
  Preparing metadata (setup.py) ... done
Note: you may need to restart the kernel to use updated packages.


### Comparing Feature Extraction Models
- mobilenet
- resnet
- densenet
- efficientnet
- vgg19

In [3]:
# Common imports
import numpy as np
import time
import os
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model

# Each model's specific import and preprocess_input
from tensorflow.keras.applications import mobilenet, resnet, densenet, efficientnet, vgg19
# from keras_squeezenet import SqueezeNet

# Function to load and preprocess the image
def load_and_preprocess_image(img_path, model_name):
    target_size = (224, 224)  # default target size for most models
    img = image.load_img(img_path, target_size=target_size)
    img = image.img_to_array(img)
    
    # Preprocess input based on model
    if model_name == 'mobilenet':
        img = mobilenet.preprocess_input(img)
    elif model_name == 'resnet':
        img = resnet.preprocess_input(img)
    elif model_name == 'densenet':
        img = densenet.preprocess_input(img)
    elif model_name == 'efficientnet':
        img = efficientnet.preprocess_input(img)
    elif model_name == 'vgg19':
        img = vgg19.preprocess_input(img)
    
    img = np.expand_dims(img, axis=0)  # Add batch dimension
    return img

# Function to create a model for feature extraction
def create_feature_model(model_name):
    if model_name == 'mobilenet':
        base_model = mobilenet.MobileNet(weights='imagenet', include_top=False)
    elif model_name == 'resnet':
        base_model = resnet.ResNet50(weights='imagenet', include_top=False)
    elif model_name == 'densenet':
        base_model = densenet.DenseNet121(weights='imagenet', include_top=False)
    elif model_name == 'efficientnet':
        base_model = efficientnet.EfficientNetB0(weights='imagenet', include_top=False)
    elif model_name == 'vgg19':
        base_model = vgg19.VGG19(weights='imagenet', include_top=False)
    # Pooling
    # x = base_model.output
    # x = GlobalAveragePooling2D()(x)
    # feature_model = Model(inputs=base_model.input, outputs=x)
    feature_model = Model(inputs=base_model.input, outputs=base_model.output)
    return feature_model

# Feature extraction for an image
def extract_features(img_path, feature_model, model_name):
    img = load_and_preprocess_image(img_path, model_name)
    features = feature_model.predict(img)
    return features


In [4]:
img_dir = "dataset/objects/train_2562"  
test_image_paths = [os.path.join(img_dir, f) for f in os.listdir(img_dir)[:10]]  

model_names = ['mobilenet', 'resnet', 'densenet', 'efficientnet', 'vgg19']
times = {}

for model_name in model_names:
    feature_model = create_feature_model(model_name)
    
    start_time = time.time()
    for img_path in test_image_paths:
        _ = extract_features(img_path, feature_model, model_name)
    end_time = time.time()
    
    duration = end_time - start_time
    times[model_name] = duration
    print(f"{model_name} took {duration:.2f} seconds")

# Compare speeds
for model_name, duration in times.items():
    print(f"{model_name}: {duration:.2f} seconds")

# Find fastest model
fastest_model = min(times, key=times.get)
print(f"The fastest model is {fastest_model} with {times[fastest_model]:.2f} seconds.")

2023-11-30 18:19:12.501938: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-11-30 18:19:12.516589: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-11-30 18:19:12.516743: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-11-30 18:19:12.517144: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

1/1 [==============================] - 0s 10ms/step


2023-11-30 18:19:14.423213: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-11-30 18:19:14.449018: I tensorflow/stream_executor/cuda/cuda_blas.cc:1614] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


1/1 [==============================] - 0s 11ms/step
mobilenet took 1.48 seconds
1/1 [==============================] - 0s 14ms/step
resnet took 0.82 seconds
1/1 [==============================] - 0s 17ms/step
densenet took 1.30 seconds
1/1 [==============================] - 0s 13ms/step
efficientnet took 0.85 seconds
1/1 [==============================] - 0s 11ms/step
vgg19 took 0.58 seconds
mobilenet: 1.48 seconds
resnet: 0.82 seconds
densenet: 1.30 seconds
efficientnet: 0.85 seconds
vgg19: 0.58 seconds
The fastest model is vgg19 with 0.58 seconds.


### Using MobileNet

In [5]:
from tensorflow.keras.applications import mobilenet_v2
model = mobilenet_v2.MobileNetV2(weights='imagenet', include_top=False, pooling='avg') 

## Feature Extraction

In [6]:
def extract_mobilenet_features(image_path, model):
    # Load and preprocess the image
    img = image.load_img(image_path, target_size=(224, 224))  # size 224x224
    img = image.img_to_array(img)
    img = mobilenet_v2.preprocess_input(img)
    img = np.expand_dims(img, axis=0)

    # Extract features
    features = model.predict(img)

    return features

In [7]:
img_dir = "dataset/objects/train_2562"

features_dict = {}

# Assuming that os.listdir(img_dir) returns a list of image filenames in img_dir
for img_name in os.listdir(img_dir):
    image_path = os.path.join(img_dir, img_name)
    if image_path.lower().endswith(('.png', '.jpg', '.jpeg')):  # Check to ensure only images are processed
        features_dict[img_name] = extract_mobilenet_features(image_path, model)

1/1 [==============================] - 0s 11ms/step


### Dimensionality Reduction

In [8]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [9]:
# Flatten features (if not already flattened by global average pooling -> pooling='avg')
feature_vectors = np.array([features_dict[img_name].flatten() for img_name in features_dict.keys()])

# Normalize features
scaler = StandardScaler()
feature_vectors_normalized = scaler.fit_transform(feature_vectors)

# Perform PCA for dimensionality reduction
pca = PCA(n_components=0.9)  # Keep 90% of the variance
principal_components = pca.fit_transform(feature_vectors_normalized)

In [10]:
print(principal_components.shape)

(158, 89)


### Comparing Dimensionality Reduction

- PCA is designed for variance maximization and dimensionality reduction, so it naturally fits the goal of retaining a high percentage of the variance.
- t-SNE does not retain variance in the same way PCA does. It's more suitable for visualization in two or three dimensions and does not provide an explained variance ratio.
- UMAP is generally used for visualization and dimensionality reduction while preserving the data's local and global structure. It's a non-linear technique, which means it may reveal structures hidden to PCA but does not directly provide an explained variance ratio.

In [11]:
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import umap.umap_ as umap
from sklearn.preprocessing import StandardScaler

# Assuming feature_vectors_normalized from the previous code block after StandardScaler

# PCA Dimensionality Reduction
pca = PCA(0.9)  # Keep 90% of the variance
pca_result = pca.fit_transform(feature_vectors_normalized)

# t-SNE Dimensionality Reduction - It does not have variance ratio
# t-SNE is not typically used for variance retention comparison, and it's a stochastic method.
# Therefore, it is more about visualization than retaining variance.
# We perform t-SNE on PCA output to speed up the process, this is optional but recommended for large datasets
tsne_result = TSNE(n_components=2, learning_rate='auto', init='pca').fit_transform(pca_result)

# UMAP Dimensionality Reduction - It also doesn't work with explained variance
# Instead, it works with neighbor graphs to preserve local and global structure.
umap_result = umap.UMAP(n_neighbors=15, min_dist=0.1, n_components=3).fit_transform(feature_vectors_normalized)

# Now, let's compare the shapes
print('Original feature size:', feature_vectors_normalized.shape[0], feature_vectors_normalized.shape[1])
print('PCA reduced feature size:', pca_result.shape[0], pca_result.shape[1])
print('t-SNE reduced feature size (not directly comparable):', tsne_result.shape[0], tsne_result.shape[1])
print('UMAP reduced feature size:', umap_result.shape[0], umap_result.shape[1])

# Print the amount of variance that each component contributes
print('PCA explained variance ratio:', pca.explained_variance_ratio_)

# If you want to also check the actual size of the reduced data you can compare the sum of the explained_variance_ratio_
print('PCA cumulative explained variance:', np.sum(pca.explained_variance_ratio_))


/home/zach/miniconda3/envs/tf/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Original feature size: 158 1280
PCA reduced feature size: 158 89
t-SNE reduced feature size (not directly comparable): 158 2
UMAP reduced feature size: 158 3
PCA explained variance ratio: [0.09139818 0.05472656 0.04616167 0.03900347 0.03329309 0.02956323
 0.02636363 0.02448547 0.02229512 0.02118651 0.02073862 0.01879155
 0.016582   0.01555348 0.01474516 0.01466676 0.01391463 0.01273887
 0.01203309 0.01154856 0.01133615 0.01105017 0.0107749  0.01016405
 0.00970528 0.00938504 0.00897842 0.00892542 0.00862486 0.00833179
 0.00818484 0.00790672 0.0077444  0.00738372 0.00707304 0.0068808
 0.0068574  0.00664174 0.00655957 0.00637274 0.00617264 0.00608117
 0.00594537 0.00580664 0.0055973  0.00545077 0.0052107  0.00508988
 0.00503997 0.00499805 0.00493937 0.00486348 0.00468501 0.00463246
 0.00451308 0.00436075 0.00431179 0.00423149 0.00418855 0.00405581
 0.00396753 0.00392062 0.00386195 0.00379205 0.003731   0.00369059
 0.00366505 0.00362174 0.00352538 0.00346725 0.00335222 0.00330679
 0.003295

In [12]:
import umap
import numpy as np

feature_list = [features for features in features_dict.values()]
feature_matrix = np.array(feature_list).reshape(len(feature_list), -1) # Reshape to (n_samples, n_features) if necessary

# Apply UMAP
reducer = umap.UMAP(n_neighbors=15, n_components=2, metric='euclidean', random_state=42)
embedding = reducer.fit_transform(feature_matrix)

reduced_features = {name: emb for name, emb in zip(features_dict.keys(), embedding)}

/home/zach/miniconda3/envs/tf/lib/python3.10/site-packages/umap/umap_.py:1943: UserWarning: n_jobs value -1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")


### Comparing Clustering Algorithms
- DBSCAN (Density-Based Spatial Clustering of Applications with Noise): It can find arbitrarily shaped clusters and can have a notion of noise, which are points that don't fit well into any cluster.
- HDBSCAN (Hierarchical DBSCAN): An extension of DBSCAN that converts it into a hierarchical clustering algorithm and enables it to find clusters of varying densities, which DBSCAN cannot do.
- Mean Shift: It doesn't require the specification of the number of clusters, as it automatically finds clusters based on data density. However, it does require bandwidth parameter selection, which indirectly influences the number of clusters.
- Affinity Propagation: It uses message passing between data points to create clusters based on their similarity. It does not require the number of clusters to be specified and can yield a varying number of clusters based on the input data and preferences parameter.

In [13]:
from sklearn.cluster import DBSCAN, MeanShift, estimate_bandwidth, AffinityPropagation
import hdbscan
from sklearn import metrics

# 'reduced_features' to list of feature coordinates for clustering
X = np.array(list(reduced_features.values()))

# DBSCAN
db = DBSCAN(eps=0.5, min_samples=5).fit(X)
db_labels = db.labels_

# HDBSCAN
hdb = hdbscan.HDBSCAN(min_cluster_size=5).fit(X)
hdb_labels = hdb.labels_

# Mean Shift
bandwidth = estimate_bandwidth(X, quantile=0.2)
ms = MeanShift(bandwidth=bandwidth, bin_seeding=True).fit(X)
ms_labels = ms.labels_

# Affinity Propagation
af = AffinityPropagation().fit(X)
af_labels = af.labels_

# Calculating the number of clusters for each algorithm
n_clusters_db = len(set(db_labels)) - (1 if -1 in db_labels else 0)
n_clusters_hdb = len(set(hdb_labels)) - (1 if -1 in hdb_labels else 0)
n_clusters_ms = len(set(ms_labels))
n_clusters_af = len(set(af_labels))

print(f"Estimated number of clusters for DBSCAN: {n_clusters_db}")
print(f"Estimated number of clusters for HDBSCAN: {n_clusters_hdb}")
print(f"Estimated number of clusters for Mean Shift: {n_clusters_ms}")
print(f"Estimated number of clusters for Affinity Propagation: {n_clusters_af}")

# measure of how similar an object is to its own cluster (cohesion) compared to other clusters (separation)
silhouette_db = metrics.silhouette_score(X, db_labels) if n_clusters_db > 1 else -1
silhouette_hdb = metrics.silhouette_score(X, hdb_labels) if n_clusters_hdb > 1 else -1
silhouette_ms = metrics.silhouette_score(X, ms_labels) if n_clusters_ms > 1 else -1
silhouette_af = metrics.silhouette_score(X, af_labels) if n_clusters_af > 1 else -1

print(f"Silhouette Coefficient for DBSCAN: {silhouette_db:.3f}")
print(f"Silhouette Coefficient for HDBSCAN: {silhouette_hdb:.3f}")
print(f"Silhouette Coefficient for Mean Shift: {silhouette_ms:.3f}")
print(f"Silhouette Coefficient for Affinity Propagation: {silhouette_af:.3f}")


Estimated number of clusters for DBSCAN: 5
Estimated number of clusters for HDBSCAN: 6
Estimated number of clusters for Mean Shift: 4
Estimated number of clusters for Affinity Propagation: 9
Silhouette Coefficient for DBSCAN: 0.495
Silhouette Coefficient for HDBSCAN: 0.358
Silhouette Coefficient for Mean Shift: 0.544
Silhouette Coefficient for Affinity Propagation: 0.492


In [14]:
import os
import shutil

cluster_dir_base = os.path.join(img_dir, 'clusters')
os.makedirs(cluster_dir_base, exist_ok=True)

inventory_count = {}

for img_name, cluster_label in zip(features_dict.keys(), db_labels):
    if cluster_label == -1:
        # -1 for noise points
        cluster_path = os.path.join(cluster_dir_base, 'noise')
    else:
        cluster_path = os.path.join(cluster_dir_base, f'cluster_{cluster_label}')
    
    os.makedirs(cluster_path, exist_ok=True)
    
    source = os.path.join(img_dir, img_name)
    destination = os.path.join(cluster_path, img_name)
    shutil.copy(source, destination)
    
    inventory_count[cluster_label] = inventory_count.get(cluster_label, 0) + 1

# Inventory count for each cluster
print("Inventory count for each object type (cluster):")
for cluster_label, count in inventory_count.items():
    if cluster_label == -1:
        print(f"Noise (unclustered) instances: {count}")
    else:
        print(f"Object type (Cluster {cluster_label}) instances: {count}")


Inventory count for each object type (cluster):
Object type (Cluster 0) instances: 30
Object type (Cluster 1) instances: 34
Noise (unclustered) instances: 11
Object type (Cluster 2) instances: 25
Object type (Cluster 3) instances: 20
Object type (Cluster 4) instances: 38
